In [1]:
# To install the official MySQL Connector for Python, please use the name mysql-connector-python:
# pip install mysql-connector-python
import mysql.connector
import datetime
import pandas as pd

In [2]:
db_connection = mysql.connector.connect(user='scott', password='tiger',
                                        host='127.0.0.1',
                                        database='saa',
                                        use_pure=False)

In [ ]:
# df = pd.read_sql('SELECT * FROM Full', con=db_connection)
cols = ['doi', 'is_oa']
df = pd.read_csv(r'/data/unpaywall_snapshot_2018-09-24T232615.csv', usecols = cols)

In [5]:
df

NameError: name 'df' is not defined

In [2]:
cnx = mysql.connector.connect(user='scott', password='tiger',
                              host='127.0.0.1',
                              database='saa',
                              use_pure=False)
cursor = cnx.cursor()

In [4]:
# query = ("SELECT doi, is_oa FROM Snapshot100")
# cursor.execute(query)

In [35]:
query = ("SELECT doi, is_oa FROM Snapshot100 "
         "WHERE doi IN ('10.1080/21645515.2017.1330236', '10.1088/0004-6256/135/4/1201')")
print(query)
cursor.execute(query)

SELECT doi, is_oa FROM Snapshot100 WHERE doi IN ('10.1080/21645515.2017.1330236', '10.1088/0004-6256/135/4/1201')


In [9]:
for (doi, is_oa) in cursor:
    print(doi, is_oa)

10.1080/21645515.2017.1330236 True
10.1088/0004-6256/135/4/1201 True


In [4]:
print ('Start Time: ' + str(datetime.datetime.now()))
query = ("SELECT doi, is_oa FROM Full "
        "WHERE doi IN ('10.1080/21645515.2017.1330236', '10.1088/0004-6256/135/4/1201')")
cursor.execute(query)
print ('Finish Time: ' + str(datetime.datetime.now()))

Start Time: 2019-03-18 21:23:31.290047
Finish Time: 2019-03-18 21:24:49.544944


In [5]:
result_set = cursor.fetchall()
for row in result_set:
    print(row)

('10.1080/21645515.2017.1330236', 'True')
('10.1088/0004-6256/135/4/1201', 'True')


In [3]:
sils_doi = ['10.1145/3173574.3173935',
            '10.1108/ILS-06-2018-0049',
            '10.1016/j.lisr.2017.03.001',
            '10.1007/s10803-017-3127-z',
            '10.1145/2854946.2854979',
            '10.1016/j.envsoft.2015.12.010',
            '10.1177%2F2053951715608877',
            '10.2218/ijdc.v10i1.352',
            '10.1145/3209624',
            '10.1016/j.lisr.2017.03.005',
            '10.1109/TVCG.2018.2864885',
            '10.1177%2F016555150202800601',
            '10.1109/TMM.2016.2614220',
            '10.1353/at.2006.0012',
            '10.1007/s10900-018-0547-4',
            '10.1108/JD-03-2016-0030',
            '10.1145/2998181.2998204',
            '10.1145/3209280.3209530',
            '10.1080/10447318.2017.1365459',
            '10.1145/3204461',
            '10.1002/asi.23124',
            '10.1002/asi.23839',
            '10.3389/fncom.2012.00019'
            '10.5210/ojphi.v6i2.5469',
            '10.1093/jamia/ocy013',
            '10.1108/JD-06-2017-0095',
            '10.1016/j.ijinfomgt.2018.07.004']

In [4]:
str1 = '\',\''.join(sils_doi)
print (str1)

10.1145/3173574.3173935','10.1108/ILS-06-2018-0049','10.1016/j.lisr.2017.03.001','10.1007/s10803-017-3127-z','10.1145/2854946.2854979','10.1016/j.envsoft.2015.12.010','10.1177%2F2053951715608877','10.2218/ijdc.v10i1.352','10.1145/3209624','10.1016/j.lisr.2017.03.005','10.1109/TVCG.2018.2864885','10.1177%2F016555150202800601','10.1109/TMM.2016.2614220','10.1353/at.2006.0012','10.1007/s10900-018-0547-4','10.1108/JD-03-2016-0030','10.1145/2998181.2998204','10.1145/3209280.3209530','10.1080/10447318.2017.1365459','10.1145/3204461','10.1002/asi.23124','10.1002/asi.23839','10.3389/fncom.2012.0001910.5210/ojphi.v6i2.5469','10.1093/jamia/ocy013','10.1108/JD-06-2017-0095','10.1016/j.ijinfomgt.2018.07.004


In [44]:
query = ("SELECT doi, is_oa FROM Snapshot100 "
        "WHERE doi IN (\'{}\')").format(str1)
print (query)
cursor.execute(query)

SELECT doi, is_oa FROM Snapshot100 WHERE doi IN ('10.1145/3173574.3173935','10.1108/ILS-06-2018-0049','10.1016/j.lisr.2017.03.001','10.1007/s10803-017-3127-z','10.1145/2854946.2854979','10.1016/j.envsoft.2015.12.010','10.1177%2F2053951715608877','10.2218/ijdc.v10i1.352','10.1145/3209624','10.1016/j.lisr.2017.03.005','10.1109/TVCG.2018.2864885','10.1177%2F016555150202800601','10.1109/TMM.2016.2614220','10.1353/at.2006.0012','10.1007/s10900-018-0547-4','10.1108/JD-03-2016-0030','10.1145/2998181.2998204','10.1145/3209280.3209530','10.1080/10447318.2017.1365459','10.1145/3204461','10.1002/asi.23124','10.1002/asi.23839','10.3389/fncom.2012.0001910.5210/ojphi.v6i2.5469','10.1093/jamia/ocy013','10.1108/JD-06-2017-0095','10.1016/j.ijinfomgt.2018.07.004')


In [5]:
print ('Start Time: ' + str(datetime.datetime.now()))
query = ("SELECT doi, is_oa FROM tmp "
        "WHERE doi IN (\'{}\')").format(str1)
cursor.execute(query)
print ('Finish Time: ' + str(datetime.datetime.now()))

Start Time: 2019-03-22 10:43:39.716006
Finish Time: 2019-03-22 10:43:39.728706


In [6]:
result_set = cursor.fetchall()
for row in result_set:
    print(row)

('10.1002/asi.23124', 'False')
('10.1002/asi.23839', 'True')
('10.1007/s10803-017-3127-z', 'False')
('10.1007/s10900-018-0547-4', 'False')
('10.1016/j.envsoft.2015.12.010', 'True')
('10.1016/j.ijinfomgt.2018.07.004', 'False')
('10.1016/j.lisr.2017.03.001', 'False')
('10.1016/j.lisr.2017.03.005', 'False')
('10.1080/10447318.2017.1365459', 'False')
('10.1093/jamia/ocy013', 'True')
('10.1108/jd-03-2016-0030', 'False')
('10.1108/jd-06-2017-0095', 'False')
('10.1109/tmm.2016.2614220', 'True')
('10.1109/tvcg.2018.2864885', 'False')
('10.1145/2854946.2854979', 'True')
('10.1145/2998181.2998204', 'False')
('10.1145/3173574.3173935', 'True')
('10.1145/3204461', 'False')
('10.1353/at.2006.0012', 'False')
('10.2218/ijdc.v10i1.352', 'True')
